In [ ]:
import numpy as np
from scipy.stats import cauchy

def fitness(individual):
    return np.sum(individual)

def weighted_lehmar_mean(succeding_rates, individual_fitnesses, trial_fitnesses):
    improvements = individual_fitnesses - trial_fitnesses
    total_improvement = np.sum(improvements)
    weights = improvements / total_improvement

    lehmar_mean = np.sum(
        weights * succeding_rates**2
    ) / np.sum(weights * succeding_rates)

    return lehmar_mean
    

problem_dimensionality = 2
generation_number = 1
initial_population_size = 100
population_size = initial_population_size
archive_size = initial_population_size
archive = np.zeros((archive_size, problem_dimensionality))
archive_fitnesses = np.empty(archive_size)
archive_fitnesses.fill(np.nan)

lower_boundary = 0 # per ora keep boundary fixed for all dimensions
upper_boundary = 1 # per ora keep boundary fixed for all dimensions

memory_size = 10 # H
memory = np.zeros((memory_size, 2)) + 0.5

p = 0.2

MEMORY_MCR_INDEX = 0
MEMORY_MMR_INDEX = 1

previous_gen_population = np.random.uniform(
    lower_boundary,
    upper_boundary,
    size=(initial_population_size, problem_dimensionality)
)


max_generations = 1000

crossing_rates = np.zeros(population_size)
mutation_rates = np.zeros(population_size)

previous_gen_individuals_fitnesses = np.array([fitness(individual) for individual in previous_gen_population])

memory_index_to_update = 0

while generation_number <= max_generations:
    
    succeding_crossover_rates = np.empty(population_size)
    succeding_crossover_rates.fill(np.nan)  
    succeding_mutation_rates = np.empty(population_size)
    succeding_mutation_rates.fill(np.nan)
    succeding_trials = np.empty((population_size, problem_dimensionality))
    succeding_trial_fitnesses = np.empty(population_size)
    succeding_trials_count = 0
    trail_individuals = np.empty((population_size, problem_dimensionality))
    trial_fitnesses = np.empty(population_size)
    are_individuals_surpassed = np.zeros(population_size)
    next_gen_individuals = previous_gen_population.copy()
    next_gen_individuals_fitnesses = previous_gen_individuals_fitnesses.copy()

    for individual_index in range(population_size):
        memory_index = np.random.randint(0, memory_size)
        if np.isnan(memory[memory_index, MEMORY_MCR_INDEX]):
            crossing_rates[individual_index] = 0
        else:
            crossing_rates[individual_index] = np.clip(
                np.random.normal(memory[memory_index, MEMORY_MCR_INDEX], 0.1),
                0,
                1.0
            )

        generated_mutation_rate = 0
        while generated_mutation_rate <= 0:
            generated_mutation_rate = np.clip(
                cauchy.rvs(memory[memory_index, MEMORY_MMR_INDEX],0.1),
                None,
                1.0
            )
        mutation_rates[individual_index] = generated_mutation_rate

        number_of_best_individuals = round(population_size * p)
        random_best_individual_index = np.random.randint(
            0, number_of_best_individuals
        )

        best_individuals_indices = np.argpartition(
            previous_gen_individuals_fitnesses, number_of_best_individuals
        )[:number_of_best_individuals]

        best_individual = previous_gen_population[
            best_individuals_indices[random_best_individual_index]
        ]

        number_individuals_in_archive = np.sum(~np.isnan(archive_fitnesses))
        individual_indices_not_current = [
            index for index in range(population_size + number_individuals_in_archive)
            if index != individual_index
        ]
        random_individual_indices =np.random.choice(
            individual_indices_not_current, 2, replace=False
        )

        random_individual_1 = (
            previous_gen_population[random_individual_indices[0]]
            if random_individual_indices[0] < population_size
            else archive[random_individual_indices[0] - population_size]
        )
        random_individual_2 = (
            previous_gen_population[random_individual_indices[1]]
            if random_individual_indices[1] < population_size
            else archive[random_individual_indices[1] - population_size]
        )
        current_individual = previous_gen_population[individual_index]

        mutant = current_individual + mutation_rates[individual_index] * (
            best_individual - current_individual
        ) + mutation_rates[individual_index] * (
            random_individual_1 - random_individual_2
        )

        for gene_index in range(problem_dimensionality):
            if mutant[gene_index] < lower_boundary:
                mutant[gene_index] = (
                    lower_boundary + current_individual[gene_index]
                ) / 2
            elif mutant[gene_index] > upper_boundary:
                mutant[gene_index] = (
                    upper_boundary + current_individual[gene_index]
                ) / 2

        random_gene_index_to_mutate = np.random.randint(0, problem_dimensionality)
        crossover_gene_index = np.random.rand(problem_dimensionality) < crossing_rates[individual_index]
        crossover_gene_index[random_gene_index_to_mutate] = True

        trail_individual = np.where(crossover_gene_index, mutant, current_individual)
        trail_individuals[individual_index] = trail_individual
        trial_fitnesses[individual_index] = fitness(trail_individual)

    for individual_index in range(population_size):

        if (
            trial_fitnesses[individual_index] <=
            previous_gen_individuals_fitnesses[individual_index]
        ):
            next_gen_individuals[individual_index] = trail_individual
            next_gen_individuals_fitnesses[individual_index] = trial_fitnesses[individual_index]

            if trial_fitnesses[individual_index] < previous_gen_individuals_fitnesses[individual_index]:
                if number_individuals_in_archive < archive_size:
                    archive[number_individuals_in_archive] = trail_individuals[individual_index]
                    archive_fitnesses[number_individuals_in_archive] = trial_fitnesses[individual_index]
                else:
                    random_element_index = np.random.randint(0, archive_size)
                    archive[random_element_index] = trail_individuals[individual_index]
                    archive_fitnesses[random_element_index] = trial_fitnesses[individual_index]

                succeding_mutation_rates[succeding_trials_count] = mutation_rates[individual_index]
                succeding_crossover_rates[succeding_trials_count] = crossing_rates[individual_index]

                succeding_trials[succeding_trials_count] = trail_individuals[individual_index]
                succeding_trial_fitnesses[succeding_trials_count] = trial_fitnesses[individual_index]

                are_individuals_surpassed[individual_index] = 1

                succeding_trials_count += 1

    if succeding_trials_count > 0:
        if (
            np.nanmax(succeding_mutation_rates) <= 0 or
            np.isnan(memory[memory_index_to_update, MEMORY_MCR_INDEX])
        ):
            memory[memory_index_to_update, MEMORY_MCR_INDEX] = np.nan
        else:
            memory[memory_index_to_update, MEMORY_MCR_INDEX] = weighted_lehmar_mean(
                succeding_crossover_rates,
                previous_gen_individuals_fitnesses[are_individuals_surpassed == 1],
                succeding_trial_fitnesses
            )
        
        memory[memory_index_to_update, MEMORY_MMR_INDEX] = weighted_lehmar_mean(
            succeding_mutation_rates,
            previous_gen_individuals_fitnesses[are_individuals_surpassed == 1],
            succeding_trial_fitnesses
        )

        memory_index_to_update = (memory_index_to_update + 1) % memory_size

    individual_count_next_gen = 




        


[array([0.62190415, 0.2107977 ]),
 array([0.34975056, 0.36384167]),
 array([0.8799735 , 0.01269936]),
 array([0.86244345, 0.9224718 ]),
 array([0.93578262, 0.08357621]),
 array([0.45412842, 0.45446963]),
 array([0.69720677, 0.30893156]),
 array([0.28273839, 0.21619662]),
 array([0.40770335, 0.72024286]),
 array([0.82124745, 0.08474982]),
 array([0.46631334, 0.59623755]),
 array([0.19711024, 0.73645649]),
 array([0.82500714, 0.31406838]),
 array([0.05517147, 0.03960687]),
 array([0.58685554, 0.19511639]),
 array([0.53807252, 0.56346983]),
 array([0.67784229, 0.49351018]),
 array([0.06988954, 0.6666077 ]),
 array([0.70314699, 0.60100723]),
 array([0.42210807, 0.1251188 ]),
 array([0.39059704, 0.28369416]),
 array([0.66757063, 0.65425812]),
 array([0.82693279, 0.9353253 ]),
 array([0.31636938, 0.46901627]),
 array([0.91756694, 0.9120229 ]),
 array([0.63754884, 0.38500694]),
 array([0.67025706, 0.76928181]),
 array([0.40218366, 0.80933687]),
 array([0.94689281, 0.34640526]),
 array([0.2403